In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
#from pylab import rcParams
import random
from datetime import datetime, timedelta

import seaborn as sns
#sb.set_style('darkgrid')


# ML
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [2]:
wild=pd.read_csv('WB_hack_3.csv',header=None)
wild.columns=['utc_event_time', 'utc_event_date', 'user_id', 'city_name', 'ecom_event_action', 'ecom_id', 
              'ecom_brand', 'ecom_variant', 'ecom_currency', 'ecom_price100', 'ecom_qty', 'ecom_grand_total100', 
              'os_manufacturer', 'device_type', 'traffic_src_kind', 'app_version', 'net_type', 'locale']

# EDA и очистка данных

In [3]:
wild.head(5)
#wild.iloc[100:110]

,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale
0,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,['view_item'],['BEFC96A9D6DEA3A9817BC5E5157B0A1C'],['6B0630D20C913739C8164D8208A1B13D'],['3B9A6121A509CBADC049AEABD3ED0223'],['RUB'],[231100],[1],[231100],Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
1,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,['add_to_cart'],['BEFC96A9D6DEA3A9817BC5E5157B0A1C'],['6B0630D20C913739C8164D8208A1B13D'],['3B9A6121A509CBADC049AEABD3ED0223'],['RUB'],[248500],[1],[248500],Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
2,2021-10-01 07:50:59,2021-10-01,12487687213536341095,Самара,['view_item'],['97F9B1135CF24F4CFFF4664435AEC2B9'],['FA74FF059207A1569295D3E83CA88E53'],['3B9A6121A509CBADC049AEABD3ED0223'],['RUB'],[75400],[1],[75400],samsung,smartphone,mobile,4.2.7000,3G,ru
3,2021-10-02 11:51:12,2021-10-02,12487687213536341095,Самара,['view_item'],['97F9B1135CF24F4CFFF4664435AEC2B9'],['FA74FF059207A1569295D3E83CA88E53'],['3B9A6121A509CBADC049AEABD3ED0223'],['RUB'],[74500],[1],[74500],samsung,smartphone,mobile,4.2.7000,3G,ru
4,2021-10-03 15:06:38,2021-10-03,10383386994170804641,Москва,['view_item'],['7D9A98C6E8BF033613B74252D9D24B33'],['681425F1391C8CA4026B28066C7B3B4A'],['3B9A6121A509CBADC049AEABD3ED0223'],['RUB'],[131900],[1],[131900],samsung,smartphone,mobile,4.3.1003,4G,ru


In [4]:
wild.shape

(2713346, 18)

* device_type и traffic_src_kind  всего 1 вида. Для построения модели не будем учитывать, но могут пригодиться для создания сервиса и аналитики на основании метрик пользователей
* Данные представлены за 61 день (2 мес)
* Неужели у одного бренда в большинстве случаев будет один продукт?

In [5]:
wild.nunique()

utc_event_time         1963997
utc_event_date              61
user_id                1237662
city_name                 2346
ecom_event_action          419
ecom_id                  39257
ecom_brand               36096
ecom_variant             25271
ecom_currency              242
ecom_price100            43413
ecom_qty                  4765
ecom_grand_total100      44332
os_manufacturer            272
device_type                  1
traffic_src_kind             1
app_version                 56
net_type                     7
locale                       4
dtype: int64

In [6]:
wild.net_type.value_counts()

Wi-Fi       1520746
4G           890634
3G           265236
2G            24812
Other         11884
Ethernet         33
5G                1
Name: net_type, dtype: int64

Всего 5 целевых действий
- view_item	- просмотр
- add_to_cart - добавление в корзину
- begin_checkout - оформление заказа
- purchase - покупка
- remove_from_cart - удаление из корзины

Видим, что во многих строках действия повторяются, посмотрим подробнее

In [7]:
wild['ecom_event_action'].value_counts().reset_index().head(15)

,index,ecom_event_action
0,['view_item'],2201443
1,['add_to_cart'],283371
2,['begin_checkout'],78639
3,['purchase'],50760
4,['remove_from_cart'],19816
5,"['begin_checkout','begin_checkout']",12168
6,"['purchase','purchase']",8580
7,"['begin_checkout','begin_checkout','begin_chec...",8395
8,"['begin_checkout','begin_checkout','begin_chec...",5955
9,"['purchase','purchase','purchase']",5592


In [8]:
# Пример
wild['ecom_event_action'].value_counts().reset_index()['index'][384]

"['purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchase','purchas

Видим, что дублирования возникли не просто так, а одна строка представляет корзину (покупку), которая может состоять из нескольких продуктов (+ их описания и стоимость). Можно сделать анализ по сопутствующим товарам.

In [9]:
wild.iloc[107]

utc_event_time                                       2021-10-21 14:09:23
utc_event_date                                                2021-10-21
user_id                                              9965105300024501182
city_name                                                         Москва
ecom_event_action      ['begin_checkout','begin_checkout','begin_chec...
ecom_id                ['6EB3A8836EBDF486669F65CA40FCB6D9','C0C9F2DC2...
ecom_brand             ['12D89E762536342E35B5F410066B0B45','A1311DE37...
ecom_variant           ['50D05F0ABB80F2B62C5F67F057E88D5F','3B9A6121A...
ecom_currency                                        ['RUB','RUB','RUB']
ecom_price100                                        [38900,15400,23600]
ecom_qty                                                         [1,1,1]
ecom_grand_total100                                  [38900,15400,23600]
os_manufacturer                                                   Xiaomi
device_type                                        

### Очистка данных
Столбцы,  в которых содержится информация о нескольких товарах в корзине
```Python
testw[['ecom_id','ecom_brand','ecom_variant','ecom_currency','ecom_price100','ecom_qty',
      'ecom_grand_total100']]
```

Приведем таблицу к нормальной форме, добавив номер заказа как признак

In [10]:
df=pd.DataFrame()

for label, content in wild.items():
    if label in ['ecom_event_action','ecom_id','ecom_brand','ecom_variant','ecom_currency','ecom_price100','ecom_qty',
      'ecom_grand_total100']:
        a=content.str.translate(str.maketrans({'[': '', ']': '',"'":''})).str.split(',').explode().reset_index()
        df['index']=a['index']
        df[str(label)]=a[str(label)]


columns=['utc_event_time', 'utc_event_date', 'user_id', 'city_name','os_manufacturer', 
         'device_type', 'traffic_src_kind', 'app_version', 'net_type', 'locale']
df=df.merge(wild[columns], right_index=True, left_on='index')

df.rename(columns = {'index': 'order_num_id'}, inplace = True)

In [11]:
df.head(5)

,order_num_id,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,utc_event_time,utc_event_date,user_id,city_name,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale
0,0,view_item,BEFC96A9D6DEA3A9817BC5E5157B0A1C,6B0630D20C913739C8164D8208A1B13D,3B9A6121A509CBADC049AEABD3ED0223,RUB,231100,1,231100,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
1,1,add_to_cart,BEFC96A9D6DEA3A9817BC5E5157B0A1C,6B0630D20C913739C8164D8208A1B13D,3B9A6121A509CBADC049AEABD3ED0223,RUB,248500,1,248500,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
2,2,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,75400,1,75400,2021-10-01 07:50:59,2021-10-01,12487687213536341095,Самара,samsung,smartphone,mobile,4.2.7000,3G,ru
3,3,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,74500,1,74500,2021-10-02 11:51:12,2021-10-02,12487687213536341095,Самара,samsung,smartphone,mobile,4.2.7000,3G,ru
4,4,view_item,7D9A98C6E8BF033613B74252D9D24B33,681425F1391C8CA4026B28066C7B3B4A,3B9A6121A509CBADC049AEABD3ED0223,RUB,131900,1,131900,2021-10-03 15:06:38,2021-10-03,10383386994170804641,Москва,samsung,smartphone,mobile,4.3.1003,4G,ru


Посмотрим датасет заново

In [12]:
df.shape
#df.to_csv('unstacked_df.csv',index=False)

(3450087, 19)

# Loading preprocessed

In [24]:
df=pd.read_csv('unstacked_df.csv')

Теперь видим более репрезентативные данные о брендах и продуктах, общая информация о других признаках продукта также значительно поменялась.

In [25]:
df.nunique()

order_num_id           2713346
ecom_event_action            6
ecom_id                 260335
ecom_brand               44576
ecom_variant              1001
ecom_currency                5
ecom_price100            13120
ecom_qty                    80
ecom_grand_total100      14177
utc_event_time         1963997
utc_event_date              61
user_id                1237662
city_name                 2346
os_manufacturer            272
device_type                  1
traffic_src_kind             1
app_version                 56
net_type                     7
locale                       4
dtype: int64

Видим еще один статус, проверяем, он был только в одном "заказе", строку удалим. 

In [26]:
df.ecom_event_action.value_counts()

view_item           2201443
begin_checkout       725076
add_to_cart          285813
purchase             217935
remove_from_cart      19816
view_item_list            4
Name: ecom_event_action, dtype: int64

In [27]:
df[df.ecom_event_action=='view_item_list']

,order_num_id,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,utc_event_time,utc_event_date,user_id,city_name,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale
2549807,2008027,view_item_list,B0270133170E1B886F4FA77B7E1F0EB9,7921BD71B70D139454324A845115B9E3,3B9A6121A509CBADC049AEABD3ED0223,RUB,1087300,1,1087300,2021-10-11 07:38:38,2021-10-11,9691711119514764683,NaN,Google,smartphone,mobile,4.3.1003,Wi-Fi,ru
2549808,2008027,view_item_list,D591C1459123DA94B018025503542940,0B1BC368C77D7A9B58F26B087958C231,3B9A6121A509CBADC049AEABD3ED0223,RUB,139300,1,139300,2021-10-11 07:38:38,2021-10-11,9691711119514764683,NaN,Google,smartphone,mobile,4.3.1003,Wi-Fi,ru
2549809,2008027,view_item_list,BD4B5ED1C236588A679299FC428F62C7,75ECD089955F52D949D4BB143AA0F50C957C9413B01995...,3B9A6121A509CBADC049AEABD3ED0223,RUB,138600,1,138600,2021-10-11 07:38:38,2021-10-11,9691711119514764683,NaN,Google,smartphone,mobile,4.3.1003,Wi-Fi,ru
2549810,2008027,view_item_list,3AF9EEF295A33CECA5FFACBCE2B87A94,B9D2461B6EEE90341EA1027A0FD6110B,3B9A6121A509CBADC049AEABD3ED0223,RUB,728900,1,728900,2021-10-11 07:38:38,2021-10-11,9691711119514764683,NaN,Google,smartphone,mobile,4.3.1003,Wi-Fi,ru


Проверим датасет на пустые значения. 

Пропуск есть только у **городов**: значений много, лучше не удалять, посмотрим, как можно заполнить их.

**Гипотеза:** по связанным признакам (валюта и страна) подставить самое часто встречающееся значение. 

Смотрим  распределение городов (первые 10). Есть перекос в сторону Москвы. Видим Минск. Смотрим валюту (отличных от рубля - 0.001%) и страны (не РФ - 0.05%) - Беларусь и беларуские рубли не встречаются так часто, как Минск, признаки можно вовсе удалить, они не представляют ценности. 

Города встречаются +- равномерно, заменим пустые значения также равномерно.

In [7]:
df.isnull().sum()

order_num_id                0
ecom_event_action           0
ecom_id                     0
ecom_brand                  0
ecom_variant                0
ecom_currency               0
ecom_price100               0
ecom_qty                    0
ecom_grand_total100         0
utc_event_time              0
utc_event_date              0
user_id                     0
city_name              418908
os_manufacturer             0
device_type                 0
traffic_src_kind            0
app_version                 0
net_type                    0
locale                      0
dtype: int64

In [8]:
df.city_name.value_counts().reset_index().head(10)

,index,city_name
0,Москва,644190
1,Санкт-Петербург,195692
2,Екатеринбург,157040
3,Краснодар,129786
4,Казань,98254
5,Минск,81628
6,Новосибирск,80709
7,Самара,69506
8,Нижний Новгород,44969
9,Ростов-на-Дону,44653


In [9]:
df.ecom_currency.value_counts()

RUB    3450051
BYN         26
KZT          6
AMD          3
KGS          1
Name: ecom_currency, dtype: int64

In [10]:
df.locale.value_counts()

ru       3448064
hy          2007
ru-RU         15
en-GB          1
Name: locale, dtype: int64

### Заполняем пропуски, преобразовываем данные, подготавливаем к модели

In [28]:
# using next valid observation to fill na
df.city_name.fillna(method='backfill',inplace=True) 

In [29]:
df=df.drop(df[(df.ecom_currency!='RUB')|((df.locale !='ru')&(df.locale !='ru-RU'))].index)
#df=df.drop(df[(df.ecom_event_action!='purchase')].index)

df.drop(columns=['ecom_currency', 'locale','utc_event_time','device_type','traffic_src_kind'],axis=1,inplace=True)

In [30]:
#df.drop(columns=['ecom_price100', 'ecom_qty'],axis=1,inplace=True)
#df

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217916 entries, 99 to 3450077
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   order_num_id         217916 non-null  int64 
 1   ecom_event_action    217916 non-null  object
 2   ecom_id              217916 non-null  object
 3   ecom_brand           217916 non-null  object
 4   ecom_variant         217916 non-null  object
 5   ecom_price100        217916 non-null  int64 
 6   ecom_qty             217916 non-null  int64 
 7   ecom_grand_total100  217916 non-null  int64 
 8   utc_event_date       217916 non-null  object
 9   user_id              217916 non-null  uint64
 10  city_name            217916 non-null  object
 11  os_manufacturer      217916 non-null  object
 12  app_version          217916 non-null  object
 13  net_type             217916 non-null  object
dtypes: int64(4), object(9), uint64(1)
memory usage: 24.9+ MB


In [31]:
# меняем типы данных 
df.ecom_price100=pd.to_numeric(df.ecom_price100)
df.ecom_qty=pd.to_numeric(df.ecom_qty)
df.ecom_grand_total100=pd.to_numeric(df.ecom_grand_total100)
df.user_id.apply(str)
df.order_num_id.apply(str)
df.utc_event_date=pd.to_datetime(df.utc_event_date)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217916 entries, 99 to 3450077
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_num_id         217916 non-null  int64         
 1   ecom_event_action    217916 non-null  object        
 2   ecom_id              217916 non-null  object        
 3   ecom_brand           217916 non-null  object        
 4   ecom_variant         217916 non-null  object        
 5   ecom_price100        217916 non-null  int64         
 6   ecom_qty             217916 non-null  int64         
 7   ecom_grand_total100  217916 non-null  int64         
 8   utc_event_date       217916 non-null  datetime64[ns]
 9   user_id              217916 non-null  uint64        
 10  city_name            217916 non-null  object        
 11  os_manufacturer      217916 non-null  object        
 12  app_version          217916 non-null  object        
 13  net_type    

In [42]:
views=df[df.ecom_event_action=='view_item'][['ecom_id','ecom_brand','utc_event_date','ecom_event_action']]\
.groupby(['ecom_id','ecom_brand','utc_event_date']).count().reset_index()

purchases=df[df.ecom_event_action=='purchase'][['ecom_id','ecom_brand','utc_event_date','ecom_event_action']]\
.groupby(['ecom_id','ecom_brand','utc_event_date']).count().reset_index()
purchases.columns=['ecom_id','ecom_brand','utc_event_date','purchases']

In [45]:
pivoted_conv=views.merge(purchases, how='left',on=['ecom_id','ecom_brand','utc_event_date'])

In [46]:
pivoted_conv.head(1)

,ecom_id,ecom_brand,utc_event_date,ecom_event_action,purchases
0,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-09-01,121,2.0


In [49]:
pivoted_conv.purchases.fillna(0)
pivoted_conv['conversion']=pivoted_conv.purchases/pivoted_conv.ecom_event_action

In [51]:
pivoted_conv.sort_values(by='conversion')

,ecom_id,ecom_brand,utc_event_date,ecom_event_action,purchases,conversion
1273,AC688D3B8E8D812E2B49496BE59286E3,30DA0D5255DF3A01E3DA47B5BC5CB3DF,2021-10-02,492,1.0,0.002033
2262,EEF2D5D6555A6E8293D2F160F3AFAB7A,911ECE2EEA64EE2B1C99B21DA2525328,2021-10-15,444,1.0,0.002252
1509,B145F002859D43DAB3C642ABD144EBD6,BFB61B83F1DB363D5F07DDEF8E522AE6,2021-09-24,326,1.0,0.003067
43,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-10-14,324,1.0,0.003086
1298,AC688D3B8E8D812E2B49496BE59286E3,30DA0D5255DF3A01E3DA47B5BC5CB3DF,2021-10-27,936,3.0,0.003205
...,...,...,...,...,...,...
2326,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-21,23,NaN,NaN
2328,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-23,12,NaN,NaN
2330,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-25,29,NaN,NaN
2331,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-26,27,NaN,NaN


In [ ]:
pivoted_conv[]

In [53]:
sales=df[df.ecom_event_action=='purchase'][['ecom_id','ecom_brand','utc_event_date','ecom_grand_total100']]\
.groupby(['ecom_id','ecom_brand','utc_event_date']).sum().reset_index()

In [58]:
pivoted_conv=pivoted_conv.merge(sales, how='left',on=['ecom_id','ecom_brand','utc_event_date'])

In [59]:
pivoted_conv.ecom_grand_total100=pivoted_conv.ecom_grand_total100/100

In [66]:
pivoted_conv=pivoted_conv.fillna(0)

In [73]:
pivoted_conv[pivoted_conv.conversion<0.1]

,ecom_id,ecom_brand,utc_event_date,ecom_event_action,purchases,conversion,ecom_grand_total100
0,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-09-01,121,2.0,0.016529,3933.0
1,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-09-02,196,3.0,0.015306,5888.0
2,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-09-03,128,2.0,0.015625,3979.0
3,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-09-04,98,0.0,0.000000,0.0
4,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-09-05,75,1.0,0.013333,2001.0
...,...,...,...,...,...,...,...
2330,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-25,29,0.0,0.000000,0.0
2331,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-26,27,0.0,0.000000,0.0
2333,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-28,153,2.0,0.013072,466.0
2334,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-29,34,0.0,0.000000,0.0


In [71]:
pivoted_conv.conversion.value_counts(bins=4)

(-0.001385, 0.0962]    2109
(0.0962, 0.192]         222
(0.192, 0.288]            3
(0.288, 0.385]            3
Name: conversion, dtype: int64

## SAMPLE EXAMPLES FOR CONVERSION

In [77]:
pivoted_conv[(pivoted_conv.ecom_id=='02C451A73FDD41CDD05F1E33E4BC60FC')|
             (pivoted_conv.ecom_id=='3AD6487EBF165BDE4EECE355B64A80A8')|
            (pivoted_conv.ecom_id=='FC7C2D6A3572340064B2B06935AC200C')].t

,ecom_id,ecom_brand,utc_event_date,ecom_event_action,purchases,conversion,ecom_grand_total100
0,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-09-01,121,2.0,0.016529,3933.0
1,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-09-02,196,3.0,0.015306,5888.0
2,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-09-03,128,2.0,0.015625,3979.0
3,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-09-04,98,0.0,0.000000,0.0
4,02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,2021-09-05,75,1.0,0.013333,2001.0
...,...,...,...,...,...,...,...
2332,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-27,10,1.0,0.100000,239.0
2333,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-28,153,2.0,0.013072,466.0
2334,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-29,34,0.0,0.000000,0.0
2335,FC7C2D6A3572340064B2B06935AC200C,416BE461EDCFDC2330BB73645A755568,2021-10-30,306,2.0,0.006536,484.0
